In [4]:
import numpy as np 
import pandas as pd
import sys
import scipy as sc
import itertools
from auxfunc import *

In [5]:
base = 2
Nxy = np.arange(1,10).reshape(3,3)
'''

#' Given a 2-d contingency table, provide a single vector of joint entropies. 
#' @param Nxy An n-contingency matrix where n > 2
#' @param unit The logarithm to be used in working out the jentropies. Defaults to "log2".

'''

dims = Nxy.shape # dim is a tuple wiht dim[0] rows  and dim[1] columns

if(len(dims) != 2):
    sys.exit("It must be a 2 dimensions array")

Nx = Nxy.sum(axis = 1) ; Ny = Nxy.sum(axis = 0) 
Hx = sc.entropy(Nx , base = base) ; Hy = sc.entropy(Ny , base = base)
Ux = np.log2(dims[0]) ; Uy = np.log2(dims[1])
Hxy = sc.entropy(Nxy.reshape(np.prod(dims)), base  = base)
VI_P = [(Hxy-Hy),(Hxy-Hx)]

#edf = pd.DataFrame([Ux,Uy,Hx,Hy,Hxy], columns = ['Ux','Uy','Hx','Hy','Hxy'])
edf = pd.DataFrame({'Type':['X','Y'],'H_P':[Hx,Hy] , 'H_U' : [Ux,Uy] ,'DeltaH_P':[(Ux - Hx) , (Uy - Hy)] , 'M_P': [Hx - VI_P[0] , Hy - VI_P[1]] , 'VI_P': VI_P }, columns = ['Type','H_U','H_P','DeltaH_P','M_P','VI_P'])
edf = edf.append(edf.apply(lambda x: np.sum(x,axis=0),axis = 0), ignore_index=True)
#edf.loc['All'] = edf.sum(axis = 0)
edf = edf.set_index('Type')


AttributeError: module 'scipy' has no attribute 'entropy'

,H_U,H_P,DeltaH_P,M_P,VI_P
Type,,,,,
X,1.584963,1.399581,0.185381,0.007882,1.391699
Y,1.584963,1.565596,0.019366,0.007882,1.557714
XY,3.169925,2.965177,0.204748,0.015765,2.949413


In [14]:
def jentropies2d(Nxy, base = 2):

    '''

    #' Given a 2-d contingency table, provide a single vector of joint entropies. 
    #' @param Nxy An n-contingency matrix where n > 2
    #' @param unit The logarithm to be used in working out the jentropies. Defaults to "log2".

    '''

    dims = Nxy.shape # dim is a tuple wiht dim[0] rows  and dim[1] columns

    if(len(dims) != 2):
        sys.exit("It must be a 2 dimensions array")

    Nx = Nxy.sum(axis = 1) ; Ny = Nxy.sum(axis = 0) 
    Hx = sc.stats.entropy(Nx , base = base) ; Hy = sc.stats.entropy(Ny , base = base)
    Ux = np.log2(dims[0]) ; Uy = np.log2(dims[1])
    Hxy = sc.stats.entropy(Nxy.reshape(np.prod(dims)), base  = base)
    VI_P = [(Hxy-Hy),(Hxy-Hx)]

    #edf = pd.DataFrame([Ux,Uy,Hx,Hy,Hxy], columns = ['Ux','Uy','Hx','Hy','Hxy'])
    edf = pd.DataFrame({'Type':['X','Y'],'H_P':[Hx,Hy] , 'H_U' : [Ux,Uy] ,'DeltaH_P':[(Ux - Hx) , (Uy - Hy)] , 'M_P': [Hx - VI_P[0] , Hy - VI_P[1]] , 'VI_P': VI_P }, columns = ['Type','H_U','H_P','DeltaH_P','M_P','VI_P'])
    edf = edf.append(edf.apply(lambda x: np.sum(x,axis=0),axis = 0), ignore_index=True)
    #edf.loc['All'] = edf.sum(axis = 0)
    edf = edf.set_index('Type')

    return edf





def jentropies_ct(Nxy, base = 2):

    dims = Nxy.shape # dim is a tuple wiht dim[0] rows  and dim[1] columns

    if(len(dims) < 2):
        sys.exit("Cannot process joint entropies for tables with less than 2 dimensions")

    if (dims[1] < 2 or dims[2] < 2):
        stop("jentropies are not defined for distributions with a singleton domain.")

    if (len(dims) == 2):
        edf = jentropies2d(Nxy , base = base)

    else:	
        if (len(dims)==3):
            lis = list()
            for i in range((dims[0])): lis.append(jentropies2d(Nxy[i]))

        else:
            lis = list()
            Nxy = Nxy.reshape((np.prod(list(Nxy.shape[0:-2]))), Nxy.shape[-2], Nxy.shape[-1])
            for i in range((Nxy.shape[0])): lis.append(jentropies2d(Nxy[i]))

        edf = pd.concat(lis)

    return edf


In [ ]:
jen